# Análise sobre evolução da capacidade dos leitos durante a pandemia de Covid-19 no estado de Goiás

Esse projeto tem como fonte os dados obtidos através do site DataSUS, subsistema LT do sistema CNES (Cadastro Nacional de Estabelecimentos de Saúde), com intuito de responder as seguintes questões:

 - 1: Comparar distribuição geográfoca dos leitos por município antes e depois da pandemia
 - 2: Evolução do número de leitos durante os anos;
 - 3: Verificar declínio na capacidade hospitalar instalada;
 - 4: Comparar a quantidade de leitos SUS e não SUS no tempo e espaço.

 Dados obtidos através da biblioteca microdatasus, em linguagem R.

 SALDANHA, Raphael de Freitas; BASTOS, Ronaldo Rocha; BARCELLOS, Christovam. Microdatasus: pacote para download e pré-processamento de microdados do Departamento de Informática do SUS (DATASUS). Cad. Saúde Pública, Rio de Janeiro , v. 35, n. 9, e00032419, 2019 . Available from http://ref.scielo.org/dhcq3y.

Abaixo o dicionário de dados, de acordo com a documentação presente no site ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/doc/IT_CNES_1706.pdf.

| SEQ | CAMPO | TIPO E TAM | DESCRIÇÃO |
|-----|-------|------------|-----------|
| 1 | CNES | CHAR (7) | Número nacional do estabelecimento de saúde |
| 2 | CODUFMUN | CHAR (6) | Código do município do estabelecimento UF+ MUNIC (sem dígito)
| 4 | REGSAUDE | CHAR (6) | Código da região de saúde
| 5 | MICR_REG | CHAR (4) | Código da micro-região de saúde
| 6 | DISTRSAN | CHAR (4) | Código do distrito sanitário
| 7 | DISTRADM | CHAR (1) | Código do distrito administrativo
| 8 | PF_PJ | CHAR (1) | Indicador de pessoa: 1-Física 3-Jurídica
| 9 |CPF_CNPJ | CHAR (14) | CPF do Estabelecimento, caso pessoa física OU CNPJ, caso pessoa jurídica
| 10 | NIV_DEP | CHAR (1) | Grau de dependência: 1-Individual 3-Mantida
| 11 | CNPJ_MAN | CHAR (14) | CNPJ da mantenedora do Estabelecimento
| 12 | ESFERA_A | CHAR (2) | Código da esfera administrativa
| 13 | ATIVIDAD | CHAR (2) | Código da atividade de ensino
| 14 | RETENCAO | CHAR (2) | Código de retenção de tributos
| 15 | NATUREZA | CHAR (2) | Código da natureza da organização
| 16 | CLIENTEL | CHAR (2) | Código de fluxo da clientela
| 17 | TP_UNID | CHAR (2) | Tipo de unidade (Estabelecimento)
| 18 | TURNO_AT | CHAR (2) | Código de turno de atendimento
| 19 | NIV_HIER | CHAR (2) | Código do nível de hierarquia
| 20 | TERCEIRO | CHAR (1) | O estabelecimento é terceiro: 1-Sim 0-Não
| 21 | TP_LEITO | CHAR (2) | Tipo do LEITO
| 22 | CODLEITO | CHAR (2) | Especialidade do LEITO
| 23 | QT_EXIST | NUMERIC (4) | Quantidade de leitos existentes
| 24 | QT_CONTR | NUMERIC (4) | Quantidade de leitos contratados
| 25 | QT_SUS | NUMERIC (4) | Quantidade de leitos para o SUS
| 26 | QT_NSUS| CHAR(1) | Indicador de EQUIPAMENTO NÃO DISPONÍVEL para o SUS, onde: 1 = SIM 0 = NÃO
| 27 | COMPETEN | CHAR (6) | Ano e Mês de competência da informação (AAAAMM)
| 28 | NAT_JUR | CHAR (4) | Natureza Jurídica

# 0.0 Imports

In [2]:
import subprocess

import pandas as pd
import numpy  as np


## 0.1 Getting Data

In [13]:
get_data = subprocess.call("Rscript ../notebooks/get_data_datasus.R", shell=True)
get_data

Your local Internet connection seems to be ok.
DataSUS FTP server seems to be up. Starting download...
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1901.dbc'
Content type 'unknown' length 32104 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1902.dbc'
Content type 'unknown' length 32114 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1903.dbc'
Content type 'unknown' length 32215 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1904.dbc'
Content type 'unknown' length 32135 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1905.dbc'
Content type 'unknown' length 31984 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publicos/CNES/200508_/Dados/LT/LTGO1906.dbc'
Content type 'unknown' length 31848 bytes (31 KB)
trying URL 'ftp://ftp.datasus.gov.br/dissemin/publi

0

## 0.2. Loading Data

In [3]:
data_raw = pd.read_csv("../data/raw/cnes_lt-2019_2022.csv", index_col= 0)
data_raw.sample(5)

/tmp/ipykernel_4756/3053487540.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  data_raw = pd.read_csv("../data/raw/cnes_lt-2019_2022.csv", index_col= 0)


,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,NIV_HIER,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR
46406,2506661,520870,NaN,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,7,69,10,0,10,0,202005,1023
95297,2589761,520140,001,NaN,NaN,NaN,M,3,26878439000105,1,...,NaN,NaN,1,3,1,0,0,1,202110,2062
109682,2437023,521483,0009,NaN,NaN,NaN,M,3,0,3,...,NaN,NaN,6,34,1,0,1,0,202202,1244
106928,9403507,521880,001,NaN,NaN,NaN,M,3,19559480000153,1,...,NaN,NaN,1,11,3,0,1,2,202201,2062
69589,2339765,520870,NaN,NaN,NaN,NaN,M,3,1011865000116,1,...,NaN,NaN,2,44,1,0,0,1,202101,2062


In [57]:
df1 = data_raw.copy()

# 1.0. Data Description

### 1.1 Data Dimensions

In [18]:
print('Number of rows: {}' .format(df1.shape[0]))
print('Number of columns: {}' .format(df1.shape[1]))

Number of rows: 137257
Number of columns: 28


### 1.2. Data Types

In [12]:
df1.dtypes

CNES          int64
CODUFMUN      int64
REGSAUDE     object
MICR_REG     object
DISTRSAN    float64
DISTRADM     object
TPGESTAO     object
PF_PJ         int64
CPF_CNPJ      int64
NIV_DEP       int64
CNPJ_MAN      int64
ESFERA_A    float64
ATIVIDAD      int64
RETENCAO    float64
NATUREZA    float64
CLIENTEL    float64
TP_UNID       int64
TURNO_AT      int64
NIV_HIER    float64
TERCEIRO    float64
TP_LEITO      int64
CODLEITO      int64
QT_EXIST      int64
QT_CONTR      int64
QT_SUS        int64
QT_NSUS       int64
COMPETEN      int64
NAT_JUR       int64
dtype: object

In [58]:
df1['COMPETEN'] = pd.to_datetime( df1['COMPETEN'], format = '%Y%m')
df1.dtypes

CNES                 int64
CODUFMUN             int64
REGSAUDE            object
MICR_REG            object
DISTRSAN           float64
DISTRADM            object
TPGESTAO            object
PF_PJ                int64
CPF_CNPJ             int64
NIV_DEP              int64
CNPJ_MAN             int64
ESFERA_A           float64
ATIVIDAD             int64
RETENCAO           float64
NATUREZA           float64
CLIENTEL           float64
TP_UNID              int64
TURNO_AT             int64
NIV_HIER           float64
TERCEIRO           float64
TP_LEITO             int64
CODLEITO             int64
QT_EXIST             int64
QT_CONTR             int64
QT_SUS               int64
QT_NSUS              int64
COMPETEN    datetime64[ns]
NAT_JUR              int64
dtype: object

In [45]:
import csv
municipios = {}

with open('../data/external/codigo_cidades.csv') as csv_file:
    reader = csv.DictReader(csv_file)
    for row in reader:
        municipios[row['Codigo_Municipio_Completo'][:-1]] = row['Nome_Municipio']

#print(municipios)

In [65]:
#Converter o codigo para o tipo string para que o dicionario funcione
df1['CODUFMUN'] = df1['CODUFMUN'].astype('str')

# Criar um coluna com o nome da cidade atraves do codigo de municipio fornecido pelo IBGE
df1['CIDADE'] = df1['CODUFMUN'].map(municipios)
df1.sample()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,CIDADE
64514,2569841,521060,4,NaN,NaN,NaN,M,3,0,3,...,NaN,5,45,2,0,2,0,2020-11-01,1244,Itaguaru


In [63]:
df11['CODUFMUN'] = df11['CODUFMUN'].astype('str')
df11.dtypes

CNES                 int64
CODUFMUN            object
REGSAUDE            object
MICR_REG            object
DISTRSAN           float64
DISTRADM            object
TPGESTAO            object
PF_PJ                int64
CPF_CNPJ             int64
NIV_DEP              int64
CNPJ_MAN             int64
ESFERA_A           float64
ATIVIDAD             int64
RETENCAO           float64
NATUREZA           float64
CLIENTEL           float64
TP_UNID              int64
TURNO_AT             int64
NIV_HIER           float64
TERCEIRO           float64
TP_LEITO             int64
CODLEITO             int64
QT_EXIST             int64
QT_CONTR             int64
QT_SUS               int64
QT_NSUS              int64
COMPETEN    datetime64[ns]
NAT_JUR              int64
CIDADE              object
dtype: object

In [64]:
df11['CIDADE'] = df11['CODUFMUN'].map(municipios)
df11.sample()

,CNES,CODUFMUN,REGSAUDE,MICR_REG,DISTRSAN,DISTRADM,TPGESTAO,PF_PJ,CPF_CNPJ,NIV_DEP,...,TERCEIRO,TP_LEITO,CODLEITO,QT_EXIST,QT_CONTR,QT_SUS,QT_NSUS,COMPETEN,NAT_JUR,CIDADE
120553,2338424,520870,NaN,NaN,NaN,NaN,M,3,1567601000224,3,...,NaN,2,31,3,0,3,0,2022-06-01,1104,Goiânia


# 2.0. Feature Engineering

# 3.0 Data Filtering

# 4.0. EDA

# 5.0. Dashboards